<a href="https://colab.research.google.com/github/darkwingpatil/Ml_hackethons/blob/main/MiniHackethon_Rag_model.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install openai
!pip install langchain-core
!pip install langchain-openai
!pip install langchain-community
!pip install chromadb
!pip install pypdf
!pip install transformers
!pip install datasets
!pip install sentence-transformers
!pip install  faiss-cpu
!pip install text-generation
!pip install langchain_huggingface

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.4/50.4 kB 3.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 408.7/408.7 kB 16.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 389.6/389.6 kB 30.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 64.0 MB/s eta 0:00:00
  Attempting uninstall: openai
    Found existing installation: openai 1.52.2
    Uninstalling openai-1.52.2:
      Successfully uninstalled openai-1.52.2
  Attempting uninstall: langchain-core
    Found existing installation: langchain-core 0.3.13
    Uninstalling langchain-core-0.3.13:
      Successfully uninstalled langchain-core-0.3.13
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.4/2.4 MB 39.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 42.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.1/3.1 MB 69.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.5/49.5 kB 3.9 MB/s eta 0:00:00
  Attempting uninstall: SQLAlche

In [2]:
import os
from datasets import load_dataset

In [3]:
!pip install jq

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 737.4/737.4 kB 16.0 MB/s eta 0:00:00


## LOAD PDF DATA

In [4]:
from langchain_community.document_loaders import PyPDFLoader,JSONLoader
# Load PDF
pdf_loaders = [
    # Duplicate documents on purpose
    PyPDFLoader("/content/AST-1.pdf"),
    PyPDFLoader("/content/AST-2.pdf")
]

docs = []
for loader in pdf_loaders:
    docs.extend(loader.load())

## Split the documents into chunks


In [5]:
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_community.document_loaders import HuggingFaceDatasetLoader
from datasets import load_dataset
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.vectorstores import FAISS
from langchain.document_loaders import PyPDFDirectoryLoader

In [6]:
# Split
#from langchain.text_splitter import RecursiveCharacterTextSplitter
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size = 500,
    chunk_overlap = 50
)

In [7]:
splits = text_splitter.split_documents(docs)
print(len(splits))
print(len(splits[0].page_content) )
splits[0].page_content

40
439


'Module 3: AST-1  \n \nTITLE: Transitioning from Research to Production Environment \nLEARNING OBJECTIVES: \nAt the end of the experiment, you will be able to transition from the research environment to the \nproduction environment. You will understand the concept of modularization and convert the model \ndeveloped in Jupyter Notebook into different modules tailored to specific functionalities: Data \nManager, Training, Pipeline, Predict, etc.'

## Load from huggingFace dataset

In [ ]:
loader = HuggingFaceDatasetLoader(
    'WingPatil/combined_PM_rag_model_dataset',
    'text'
)

documents  = loader.load()
data_chunks = text_splitter.split_documents(documents)


In [ ]:
combined_chunks = data_chunks + splits

In [8]:
embeddings = HuggingFaceEmbeddings(model_name='sentence-transformers/all-MiniLM-L6-v2',
                                       model_kwargs={'device': 'cuda'})
# # cde-small-v1 good one

<ipython-input-8-4eb1e88c8279>:1: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFaceEmbeddings``.
  embeddings = HuggingFaceEmbeddings(model_name='sentence-transformers/all-MiniLM-L6-v2',
/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or d

modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/10.7k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

In [9]:
from langchain_community.vectorstores import Chroma # Light-weight and in memory

In [10]:
persist_directory = 'docs/chroma/'
!rm -rf ./docs/chroma  # remove old database files if any


In [11]:
vectordb = Chroma.from_documents(
    documents=splits, # splits we created earlier
    embedding=embeddings,
    persist_directory=persist_directory, # save the directory
)

In [12]:
retriever = vectordb.as_retriever(search_type="mmr",search_kwargs={"k": 7, "fetch_k":15})

In [13]:
!pip install huggingface_hub
from huggingface_hub import login

# Log in to Hugging Face
  # Replace with your actual token
!huggingface-cli login


    _|    _|  _|    _|    _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|_|_|_|    _|_|      _|_|_|  _|_|_|_|
    _|    _|  _|    _|  _|        _|          _|    _|_|    _|  _|            _|        _|    _|  _|        _|
    _|_|_|_|  _|    _|  _|  _|_|  _|  _|_|    _|    _|  _|  _|  _|  _|_|      _|_|_|    _|_|_|_|  _|        _|_|_|
    _|    _|  _|    _|  _|    _|  _|    _|    _|    _|    _|_|  _|    _|      _|        _|    _|  _|        _|
    _|    _|    _|_|      _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|        _|    _|    _|_|_|  _|_|_|_|

    To login, `huggingface_hub` requires a token generated from https://huggingface.co/settings/tokens .
Enter your token (input will not be visible): 
Add token as git credential? (Y/n) n
Token is valid (permission: write).
Your token has been saved to /root/.cache/huggingface/token
Login successful


In [14]:
from langchain_huggingface import HuggingFaceEndpoint

In [15]:
# from langchain_openai import ChatOpenAI
from langchain.llms import OpenAI


llm = HuggingFaceEndpoint(
    repo_id="meta-llama/Llama-3.2-3B-Instruct",
    task="text-generation",
    max_new_tokens = 512,
    top_k = 30,
    temperature = 0.1,
    repetition_penalty = 1.03,
)
# HuggingFaceH4/zephyr-7b-beta
# mistralai/Mistral-7B-Instruct-v0.2    too good
#  meta-llama/Llama-3.2-3B-Instruct

In [16]:
from langchain_core.prompts import PromptTemplate     #
from langchain_core.output_parsers import StrOutputParser
from langchain.schema.runnable import RunnablePassthrough

In [17]:
# Build prompt
template = """Use the following pieces of context to answer the question at the end.
If you don't know the answer, just say that you don't know, don't try to make up an answer.
Always say "thanks for asking!" at the end of the answer.
{context}
Question: {question}
Helpful Answer:"""
QA_PROMPT = PromptTemplate(input_variables=["context", "question"],template=template)

In [18]:
from langchain_core.runnables import RunnableLambda

In [19]:
rag_chain= {"context":RunnableLambda(lambda x:x["question"])| retriever,
         "question": lambda x:x["question"]}|QA_PROMPT | llm |StrOutputParser()

In [20]:
# Question 1
try:
  response=rag_chain.invoke({"question" :"Wht are several benefits which is provided by testing"})
  print(response)
except Exception as e:
  print(e)

 Thanks for asking! According to the text, several benefits of testing include minimizing potential risks such as system crashes, data loss, security vulnerabilities, or financial losses. Testing also facilitates code maintenance and refactoring, serves as documentation and understanding, and boosts testing productivity. Additionally, testing helps developers understand the codebase better by acting as living examples and serving as a guide for future development. Thanks for asking!


In [22]:
!pip install gradio

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.7/56.7 MB 14.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 319.8/319.8 kB 28.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 447.5/447.5 kB 37.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.0/11.0 MB 119.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 130.2/130.2 kB 13.6 MB/s eta 0:00:00
  Attempting uninstall: websockets
    Found existing installation: websockets 13.1
    Uninstalling websockets-13.1:
      Successfully uninstalled websockets-13.1
  Attempting uninstall: markupsafe
    Found existing installation: MarkupSafe 3.0.2
    Uninstalling MarkupSafe-3.0.2:
      Successfully uninstalled MarkupSafe-3.0.2
  Attempting uninstall: huggingface-hub
    Found existing installation: huggingface-hub 0.24.7
    Uninstalling huggingface-hub-0.24.7:
      Successfully uninstalled huggingface-hub-0.24.7


In [23]:
import gradio as gr

# Assuming retriever and QA_PROMPT are already defined

def answer_question(question):
  try:
    response = rag_chain.invoke({"question": question})
    return response
  except Exception as e:
    return f"Error: {e}"

# Create the Gradio interface
iface = gr.Interface(
    fn=answer_question,
    inputs=gr.Textbox(lines=2, placeholder="Enter your question here..."),
    outputs="text",
    title="Question Answering App",
    description="Ask a question and get an answer based on the provided context.",
)

In [24]:
iface.launch()

Running Gradio in a Colab notebook requires sharing enabled. Automatically setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://4cbacbb91b697c7111.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)


In [ ]:
import json

config = {
    "vectorstore_path": "docs/chroma/",
    "llm_endpoint": {
        "repo_id": "meta-llama/Llama-3.2-3B-Instruct",
        "task": "text-generation",
        "max_new_tokens": 512,
        "top_k": 30,
        "temperature": 0.1,
        "repetition_penalty": 1.03
    }
}

with open("config.json", "w") as f:
    json.dump(config, f)

In [ ]:
from huggingface_hub import HfApi

api = HfApi()
repo_id = "WingPatil/MiniHackethon_Research_Dev_search"  # Choose a unique model name
api.create_repo(repo_id=repo_id, exist_ok=True)

# Upload files
api.upload_folder(
    folder_path="docs/chroma/",  # Path to your Chroma vectorstore folder
    repo_id=repo_id,
    path_in_repo="chroma_store"
)
api.upload_file(
    path_or_fileobj="config.json",
    path_in_repo="config.json",
    repo_id=repo_id
)

data_level0.bin:   0%|          | 0.00/1.68M [00:00<?, ?B/s]

header.bin:   0%|          | 0.00/100 [00:00<?, ?B/s]

length.bin:   0%|          | 0.00/4.00k [00:00<?, ?B/s]

Upload 3 LFS files:   0%|          | 0/3 [00:00<?, ?it/s]

CommitInfo(commit_url='https://huggingface.co/WingPatil/MiniHackethon_Research_Dev_search/commit/1e15a423953eeb17681b72162c3febe12c733a8f', commit_message='Upload config.json with huggingface_hub', commit_description='', oid='1e15a423953eeb17681b72162c3febe12c733a8f', pr_url=None, pr_revision=None, pr_num=None)

In [ ]:
from huggingface_hub import HfApi, hf_hub_download
import os

repo_id = "WingPatil/MiniHackethon_Research_Dev_search"
local_dir = "downloaded_chroma_store"  # Directory to store downloaded files
os.makedirs(local_dir, exist_ok=True)

# List files in the repository folder
api = HfApi()
files = api.list_repo_files(repo_id=repo_id)

# Download each file in chroma_store directory
for file in files:
    if file.startswith("chroma_store/"):  # Only download files in chroma_store
        filename = os.path.basename(file)
        local_path = hf_hub_download(repo_id=repo_id, filename=file, local_dir=local_dir)
        print(f"Downloaded {filename} to {local_path}")



data_level0.bin:   0%|          | 0.00/1.68M [00:00<?, ?B/s]

Downloaded data_level0.bin to downloaded_chroma_store/chroma_store/220d3501-3b03-43f6-b516-4a21a8548b79/data_level0.bin


header.bin:   0%|          | 0.00/100 [00:00<?, ?B/s]

Downloaded header.bin to downloaded_chroma_store/chroma_store/220d3501-3b03-43f6-b516-4a21a8548b79/header.bin


length.bin:   0%|          | 0.00/4.00k [00:00<?, ?B/s]

Downloaded length.bin to downloaded_chroma_store/chroma_store/220d3501-3b03-43f6-b516-4a21a8548b79/length.bin


(…)03-43f6-b516-4a21a8548b79/link_lists.bin: 0.00B [00:00, ?B/s]

Downloaded link_lists.bin to downloaded_chroma_store/chroma_store/220d3501-3b03-43f6-b516-4a21a8548b79/link_lists.bin


chroma_store/chroma.sqlite3:   0%|          | 0.00/475k [00:00<?, ?B/s]

Downloaded chroma.sqlite3 to downloaded_chroma_store/chroma_store/chroma.sqlite3


In [ ]:
local_dir_config = "downloaded_config"

for file in files:
    if file.startswith("config"):  # Only download files in chroma_store
        filename = os.path.basename(file)
        local_path = hf_hub_download(repo_id=repo_id, filename=file, local_dir=local_dir_config)
        print(f"Downloaded {filename} to {local_path}")

.gitattributes
chroma_store/220d3501-3b03-43f6-b516-4a21a8548b79/data_level0.bin
chroma_store/220d3501-3b03-43f6-b516-4a21a8548b79/header.bin
chroma_store/220d3501-3b03-43f6-b516-4a21a8548b79/length.bin
chroma_store/220d3501-3b03-43f6-b516-4a21a8548b79/link_lists.bin
chroma_store/chroma.sqlite3
config.json
got in


config.json:   0%|          | 0.00/212 [00:00<?, ?B/s]

Downloaded config.json to downloaded_config/config.json
